In [1315]:
data_path = "data"
model = "gpt-3.5-turbo"
initial_message = [{"role": "user", "content": ""}]

In [1316]:
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown
client = OpenAI()

In [ ]:
def best(dataset):
    if dataset.iloc[-1]["Choice"] == "No":
        best_content = dataset.iloc[-1]["Instruction2"]
        best_response = dataset.iloc[-1]["Response2"]
    else:
        best_content = dataset.iloc[-1]["Instruction1"]
        best_response = dataset.iloc[-1]["Response1"]
    best_message = [{"role": "user", "content": best_content}]
    return best_message,best_response

In [1319]:
try:
    with open(data_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    best_message, best_response = best(dataset)

except FileNotFoundError:
    dataset = []
    dataset = pd.DataFrame(dataset, columns=['Instruction1', 'Response1', 'Instruction2', 'Response2', 'Sampled', 'Choice', 'Data'])
    best_message = initial_message
    best_response = client.chat.completions.create(
        messages = initial_message,
        model = model,
        ).choices[0].message.content
    
candidate_messages = [best_message]
next_candidate_messages = []
generation_distance = 0
message_in_candidate_messages = 0


In [1320]:
def battle(best_message, best_response, candidate_message):

    dataset = [{"input1": candidate_message, "input2": best_message, "completion2":best_response}]

    df = pd.DataFrame(dataset)
    df.to_json(data_path, orient="records", lines=True)

    !oaieval gpt-3.5-turbo battles --record_path logs/logs
    
    with open(json_logs_path, "r") as f:
        df = pd.read_json(f, lines=True)

    #current_time = datetime.datetime.now()
    #formatted_time = current_time.strftime("%Y%m%d_%H%M%S")
    #df.to_json(os.path.join(df_path, formatted_time), lines=True, orient="records")

    instruction1 = candidate_message[-1]["content"]
    instruction2 = best_message[-1]["content"]

    battle_prompt_content = df["data"].iloc[-2]["prompt"][0]["content"]
    response1 = battle_prompt_content.split("\n[Response 1]\n",)[1].split("\n\n[Instruction 2]\n")[0]
    response1 = response1.replace("\\'", "'").replace("\\n", "\n")
    response2 = battle_prompt_content.split("\n[Response 2]\n",)[1].split("\n\n\nIs the first response better than the second?")[0]
    response2 = response2.replace("\\'", "'").replace("\\n", "\n")
    print(f"Response1: {response1}")
    #print(f"response2: {response2}")

    sampled = df["data"].iloc[-2]["sampled"][0]

    choice = df["data"].iloc[-1]["choice"]

    data = {'Instruction1': instruction1, 'Response1': response1, 'Instruction2': instruction2, 'Response2': response2, 'Sampled': sampled, 'Choice': choice, 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = df.to_dict()
    return data

In [1321]:
def switch_system_and_user(message):
    new_message = []
    if message[0]["role"] == "system":
        new_message.append([{"role": "user", "content": message[0]["content"]}])
    else:
        new_message.append([{"role": "system", "content": message[0]["content"]}])
    return new_message

In [1322]:
def find_parents(content, dataset):
    #print (f"finding parents {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Response1"] == content]
    parents = dataset["Instruction1"].unique()
    return parents

In [1323]:
def find_children(content, dataset):
    #print (f"finding children {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Instruction1"] == content]
    children = dataset["Response1"].unique()
    return children

In [1324]:
def list_candidate_messages(dataset, best_message, generation_distance, roles):
    best_content = best_message[-1]["content"]
    last = [best_content]
    list_of_contents = [best_content]

    next = []
    for i in range(generation_distance):
        #print (f"starting level {i}, generation distance {generation_distance}")
        for content in last:
            #print(f"last_up: {last_up}, now {content}")
            next.extend(find_parents(content, dataset))
            next.extend(find_children(content, dataset))
            
        #print (f"behind for loops")
        list_of_contents.extend(next)
        last = next.copy()
        #print(f"endind level {i}")
        
    list_of_contents = pd.array(list_of_contents).unique().tolist()
    
    #best_message_index = list_of_contents.index(best_message[0][content])
    #start = max(best_message_index - generation_distance, 0)
    #stop = min(len(list_of_contents))
    #stop = len(list_of_contents) - 1

    messages = []
    for content in list_of_contents:
        if roles == "system-user" or "user":
            messages.append([{"role":"user","content":content}])
        if (roles == "system-user") or (roles == "system"):
            print("Creating message without user role.")
            messages.append([{"role":"system","content":content}])

    return messages

In [1325]:
def make_x_shot_prompt(dataset,message,num_few_shot):
    few_shot_prompt = []
    for _, key_battle in dataset[dataset["Choice"]=="Yes"].iloc[-num_few_shot:].iterrows():
        few_shot_prompt.append({"role": "system", "content": key_battle["Instruction1"], "name": "example_user"})
        few_shot_prompt.append({"role": "system", "content": key_battle["Response1"], "name": "example_assistant"})
    message = few_shot_prompt + message
    return message


In [1326]:
def user_choice_prompt(candidate_message,candidate_messages,message_in_candidate_messages,dataset, best_message, best_response, generation_distance, roles, battles_generation):

    prompt = "You are comparing two responses to the following two instructions."

    prompt += "\n\n[Instruction 1]\n"
    prompt += candidate_message[-1]["content"]

    prompt += "\n\n[Response 1]\n"
    if message_in_candidate_messages < len(candidate_messages):
        candidate_message = candidate_messages[message_in_candidate_messages]
        message_in_candidate_messages += 1
    else:
        if battles_generation == "dense":
            candidate_messages = list_candidate_messages(dataset, best_message, generation_distance, roles)
            generation_distance += 1
            candidate_message = candidate_messages[0]
            message_in_candidate_messages = 1
            print(f"candidate messages listed, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")
        else:
            print("Code not completed!")
    completion = client.chat.completions.create(
        messages = candidate_message,
        model = model,
        )
    response1 = completion.choices[0].message.content
    prompt += response1
    
    prompt += "\n\n[Instruction 2]\n"
    prompt += best_message[-1]["content"]

    prompt += "\n\n[Response 2]\n"
    prompt += best_response

    prompt += "\n\n\nIs the first response better than the second? You must provide one answer based on your subjective view."

    data = {'Instruction1': candidate_message[-1]["content"], 'Response1': response1, 'Instruction2': best_message[-1]["content"], 'Response2': best_response, 'Sampled': {}, 'Choice': "", 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = completion.to_dict()

    display_markdown(prompt, raw=True)
    return data,candidate_messages,message_in_candidate_messages,generation_distance


In [1327]:
if choices == "user":
    data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt(candidate_message,candidate_messages,message_in_candidate_messages,dataset, best_message, best_response, generation_distance, roles, battles_generation)

You are comparing two responses to the following two instructions.

[Instruction 1]


[Response 1]
Hello! How can I assist you today?

[Instruction 2]


[Response 2]
Hello! How can I assist you today?


Is the first response better than the second? You must provide one answer based on your subjective view.

In [1328]:
choice = "No"
#choice = "Yes"
data.at[0, "Choice"] = choice
print(f"Your answer:{choice}")
if choice == "Yes":
    best_message = candidate_message
    best_response = data["Response1"].iloc[0]
    print(f"New best message and response!")
    if battles_generation == "dense":
        generation_distance = 0
dataset = pd.concat([dataset, data],ignore_index=True)
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt(candidate_message,candidate_messages,message_in_candidate_messages,dataset, best_message, best_response, generation_distance, roles, battles_generation)

Your answer:No


TypeError: user_choice_prompt() missing 1 required positional argument: 'battles_generation'

In [ ]:
dataset[-5:]

In [ ]:
if choices == "ai":
    for candidate_message in candidate_messages:
        if num_few_shot > 0:
            candidate_message = make_x_shot_prompt(dataset,candidate_message,num_few_shot)
        data = battle(best_message, best_response, candidate_message)
        dataset = pd.concat([dataset, data],ignore_index=True)
        dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
        if data["Choice"].iloc[0] == "Yes":
            best_message = candidate_message
            best_response = data["Response1"].iloc[0]
            print(f"New best message:{best_message}")
            if battles_generation == "dense":
                generation_distance = 0

        print(f"Instruction1: {candidate_message[-1]["content"]}")

        if battles_generation == "only-new":
            new_message = [{"role": candidate_message[0]["role"], "content":data["Response1"].iloc[0]}]
            if new_message not in next_candidate_messages:
                next_candidate_messages.append(new_message)
                if roles == "system-user":
                    next_candidate_messages.append(switch_system_and_user(new_message))

    if battles_generation == "only-new":
        candidate_messages = next_candidate_messages
        next_candidate_messages = []
    if battles_generation == "dense":
        #print("going to list candidate messages")
        candidate_messages = list_candidate_messages(dataset, best_message, generation_distance, roles)
        generation_distance += 1
        #print("candidate messages listed")
    print(f"all done, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")

In [ ]:
for entry in dataset[dataset["Choice"]=="Yes"].iloc[-1].drop("Data"):
    display_markdown(entry, raw=True)

In [ ]:
dataset.iloc[-20:]

In [ ]:
candidate_messages

In [ ]:
dataset["Sampled"].unique()